In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv')
obesity = pd.read_csv('LakeCounty_Health_-6177935595181947989.csv')

In [3]:
df = df.dropna()

In [4]:
print("\nColumn names:")
print(df.columns.tolist())


Column names:
['OBJECTID', 'GEOID10', 'GEOID20', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'CSA', 'CSA_Name', 'CBSA', 'CBSA_Name', 'CBSA_POP', 'CBSA_EMP', 'CBSA_WRK', 'Ac_Total', 'Ac_Water', 'Ac_Land', 'Ac_Unpr', 'TotPop', 'CountHU', 'HH', 'P_WrkAge', 'AutoOwn0', 'Pct_AO0', 'AutoOwn1', 'Pct_AO1', 'AutoOwn2p', 'Pct_AO2p', 'Workers', 'R_LowWageWk', 'R_MedWageWk', 'R_HiWageWk', 'R_PCTLOWWAGE', 'TotEmp', 'E5_Ret', 'E5_Off', 'E5_Ind', 'E5_Svc', 'E5_Ent', 'E8_Ret', 'E8_off', 'E8_Ind', 'E8_Svc', 'E8_Ent', 'E8_Ed', 'E8_Hlth', 'E8_Pub', 'E_LowWageWk', 'E_MedWageWk', 'E_HiWageWk', 'E_PctLowWage', 'D1A', 'D1B', 'D1C', 'D1C5_RET', 'D1C5_OFF', 'D1C5_IND', 'D1C5_SVC', 'D1C5_ENT', 'D1C8_RET', 'D1C8_OFF', 'D1C8_IND', 'D1C8_SVC', 'D1C8_ENT', 'D1C8_ED', 'D1C8_HLTH', 'D1C8_PUB', 'D1D', 'D1_FLAG', 'D2A_JPHH', 'D2B_E5MIX', 'D2B_E5MIXA', 'D2B_E8MIX', 'D2B_E8MIXA', 'D2A_EPHHM', 'D2C_TRPMX1', 'D2C_TRPMX2', 'D2C_TRIPEQ', 'D2R_JOBPOP', 'D2R_WRKEMP', 'D2A_WRKEMP', 'D2C_WREMLX', 'D3A', 'D3AAO', 'D3AMM', 'D3A

In [5]:
# this column is a distance measure, anything negative is a null value and should be dropped
df = df[df['D4A'] > 0]

In [6]:
def fips_to_state(fips_code):
    fips_to_state_dict = {
        1: 'Alabama', 2: 'Alaska', 4: 'Arizona', 5: 'Arkansas',
        6: 'California', 8: 'Colorado', 9: 'Connecticut', 10: 'Delaware',
        11: 'District of Columbia', 12: 'Florida', 13: 'Georgia',
        15: 'Hawaii', 16: 'Idaho', 17: 'Illinois', 18: 'Indiana',
        19: 'Iowa', 20: 'Kansas', 21: 'Kentucky', 22: 'Louisiana',
        23: 'Maine', 24: 'Maryland', 25: 'Massachusetts', 26: 'Michigan',
        27: 'Minnesota', 28: 'Mississippi', 29: 'Missouri', 30: 'Montana',
        31: 'Nebraska', 32: 'Nevada', 33: 'New Hampshire', 34: 'New Jersey',
        35: 'New Mexico', 36: 'New York', 37: 'North Carolina', 38: 'North Dakota',
        39: 'Ohio', 40: 'Oklahoma', 41: 'Oregon', 42: 'Pennsylvania',
        44: 'Rhode Island', 45: 'South Carolina', 46: 'South Dakota',
        47: 'Tennessee', 48: 'Texas', 49: 'Utah', 50: 'Vermont',
        51: 'Virginia', 53: 'Washington', 54: 'West Virginia',
        55: 'Wisconsin', 56: 'Wyoming', 72: 'Puerto Rico'
    }
    
    return fips_to_state_dict.get(int(fips_code), 'Unknown')

In [7]:
# easier for me to read personally if its text
df['state'] = df['STATEFP'].apply(fips_to_state)

In [8]:
df_state = pd.DataFrame()

In [9]:
# aggregate census block data into state averages, respecting the sizes of census blocks and their population for weighting
def weighted_avg(group, stat, weight):
    return np.average(group[stat], weights = group[weight])

pop_stats = ['Pct_AO0', 'Pct_AO1', 'Pct_AO2p', 'R_PCTLOWWAGE', 'D4A', 'NatWalkInd']
for stat in pop_stats:
    df_state[stat] = df.groupby('state').apply(weighted_avg, stat, 'TotPop')
area_stats = ['D1B', 'D1C']
for stat in area_stats:
    df_state[stat] = df.groupby('state').apply(weighted_avg, stat, 'Ac_Unpr')

In [10]:
# lets us filter states into regions
def state_to_region(state_name):
    region_dict = {
        # northeast states
        'Connecticut': 'Northeast', 'Maine': 'Northeast', 'Massachusetts': 'Northeast',
        'New Hampshire': 'Northeast', 'New Jersey': 'Northeast', 'New York': 'Northeast',
        'Pennsylvania': 'Northeast', 'Rhode Island': 'Northeast', 'Vermont': 'Northeast',
        # midwest states
        'Illinois': 'Midwest', 'Indiana': 'Midwest', 'Iowa': 'Midwest',
        'Kansas': 'Midwest', 'Michigan': 'Midwest', 'Minnesota': 'Midwest',
        'Missouri': 'Midwest', 'Nebraska': 'Midwest', 'North Dakota': 'Midwest',
        'Ohio': 'Midwest', 'South Dakota': 'Midwest', 'Wisconsin': 'Midwest',
        # south states
        'Alabama': 'South', 'Arkansas': 'South', 'Delaware': 'South',
        'Florida': 'South', 'Georgia': 'South', 'Kentucky': 'South',
        'Louisiana': 'South', 'Maryland': 'South', 'Mississippi': 'South',
        'North Carolina': 'South', 'Oklahoma': 'South', 'South Carolina': 'South',
        'Tennessee': 'South', 'Texas': 'South', 'Virginia': 'South',
        'West Virginia': 'South', 'District of Columbia': 'South',
        # west states
        'Alaska': 'West', 'Arizona': 'West', 'California': 'West',
        'Colorado': 'West', 'Hawaii': 'West', 'Idaho': 'West',
        'Montana': 'West', 'Nevada': 'West', 'New Mexico': 'West',
        'Oregon': 'West', 'Utah': 'West', 'Washington': 'West',
        'Wyoming': 'West'
    }
    
    return region_dict.get(state_name, 'Unknown')

In [11]:
df_state['region'] = df_state.index.map(state_to_region)

In [12]:
obesity = obesity.set_index('NAME')

In [13]:
df_state['obesity'] = obesity['Obesity']

In [14]:
df_state

,Pct_AO0,Pct_AO1,Pct_AO2p,R_PCTLOWWAGE,D4A,NatWalkInd,D1B,D1C,region,obesity
state,,,,,,,,,,
Alabama,0.116564,0.433666,0.449769,0.294283,481.284248,13.119320,3.252453,2.624661,South,35.6
Arizona,0.085540,0.374805,0.538089,0.224504,569.848272,13.292608,7.024339,3.797911,West,28.4
Arkansas,0.102609,0.441920,0.453433,0.271414,600.272346,13.236262,2.555079,2.309643,South,34.5
California,0.080051,0.308560,0.609327,0.217197,487.763887,14.060437,5.714508,2.666763,West,24.2
Colorado,0.065718,0.328480,0.605802,0.210583,545.351323,14.005839,6.333512,3.721843,West,20.2
Connecticut,0.128951,0.378669,0.492248,0.225619,482.727127,13.045171,4.446397,2.472883,Northeast,25.3
Delaware,0.083929,0.383819,0.532251,0.246152,576.880686,13.132421,2.981325,1.791805,South,29.7
District of Columbia,0.339118,0.436367,0.218721,0.181940,269.323441,14.457063,22.616850,15.381636,South,22.1
Florida,0.085669,0.408410,0.503800,0.240355,572.516943,13.493997,5.771270,2.867115,South,26.8


In [15]:
# finding outliers using standard deviations away from the mean

In [16]:
def find_outliers(df, columns, threshold = 3):
    outliers = {}

    # check if its too many standard deviations away
    for col in columns:
        zscore = np.abs((df[col] - df[col].mean()) / df[col].std())
        outliers[col] = df.index[zscore > threshold].tolist()
    
    return outliers

In [17]:
outliers = find_outliers(df_state, pop_stats + area_stats + ['obesity'], threshold = 3)
print(outliers)

{'Pct_AO0': ['District of Columbia', 'New York'], 'Pct_AO1': ['West Virginia'], 'Pct_AO2p': ['District of Columbia'], 'R_PCTLOWWAGE': ['Mississippi'], 'D4A': ['West Virginia'], 'NatWalkInd': ['West Virginia'], 'D1B': ['District of Columbia'], 'D1C': ['District of Columbia', 'New Hampshire'], 'obesity': []}


In [18]:
# let's split our data into regions in order to examine regional means

In [19]:
northeast = df_state[df_state['region'] == 'Northeast']
south = df_state[df_state['region'] == 'South']
midwest = df_state[df_state['region'] == 'Midwest']
west = df_state[df_state['region'] == 'West']

In [20]:
northeast.mean(numeric_only=True)

Pct_AO0           0.170399
Pct_AO1           0.386023
Pct_AO2p          0.441892
R_PCTLOWWAGE      0.231460
D4A             495.377765
NatWalkInd       14.008413
D1B               6.134380
D1C               4.311214
obesity          26.400000
dtype: float64

In [21]:
south.mean(numeric_only=True)

Pct_AO0           0.115792
Pct_AO1           0.416503
Pct_AO2p          0.462362
R_PCTLOWWAGE      0.254618
D4A             559.399927
NatWalkInd       13.358573
D1B               5.882133
D1C               3.838570
obesity          31.847059
dtype: float64

In [22]:
midwest.mean(numeric_only=True)

Pct_AO0           0.106275
Pct_AO1           0.384699
Pct_AO2p          0.506355
R_PCTLOWWAGE      0.256841
D4A             496.398375
NatWalkInd       13.495667
D1B               5.133206
D1C               3.242591
obesity          31.000000
dtype: float64

In [23]:
west.mean(numeric_only=True)

Pct_AO0           0.073581
Pct_AO1           0.342497
Pct_AO2p          0.582801
R_PCTLOWWAGE      0.225116
D4A             554.499382
NatWalkInd       13.918562
D1B               5.330822
D1C               2.979670
obesity          26.433333
dtype: float64

In [24]:
northeast.std(numeric_only=True)

Pct_AO0           0.083260
Pct_AO1           0.026264
Pct_AO2p          0.074231
R_PCTLOWWAGE      0.014364
D4A             136.843876
NatWalkInd        0.543572
D1B               3.170454
D1C               3.313203
obesity           2.121320
dtype: float64

In [25]:
south.std(numeric_only=True)

Pct_AO0           0.061114
Pct_AO1           0.043886
Pct_AO2p          0.084306
R_PCTLOWWAGE      0.034717
D4A             130.395379
NatWalkInd        0.820964
D1B               4.643408
D1C               3.073113
obesity           3.785848
dtype: float64

In [26]:
midwest.std(numeric_only=True)

Pct_AO0          0.034354
Pct_AO1          0.028077
Pct_AO2p         0.059356
R_PCTLOWWAGE     0.020698
D4A             72.174509
NatWalkInd       0.427363
D1B              1.414328
D1C              0.549749
obesity          1.981918
dtype: float64

In [27]:
west.std(numeric_only=True)

Pct_AO0          0.016111
Pct_AO1          0.039941
Pct_AO2p         0.050405
R_PCTLOWWAGE     0.027536
D4A             44.581462
NatWalkInd       0.432129
D1B              1.664832
D1C              0.896099
obesity          3.067165
dtype: float64

In [28]:
# First we are going to try to predict walkability scores using economic metrics

In [29]:
X = df_state[['D1B', 'D1C', 'R_PCTLOWWAGE']]
y = df_state['NatWalkInd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 1)

X_scaled = preprocessing.StandardScaler().fit_transform(X_train)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [30]:
mae = mean_absolute_error(y_test, y_pred)
mae

0.5338902046954777

In [31]:
mse = mean_squared_error(y_test, y_pred)
mse

0.4293205433222877

In [32]:
r2 = r2_score(y_test, y_pred)
r2

0.11457620888381947

In [33]:
print(f'Model coefficients:\n{model.coef_}\n intercept:\n{model.intercept_}')

Model coefficients:
[-0.02112835  0.12222615 -1.77232035]
 intercept:
13.733648722805174


In [34]:
# the r2 is not good, and the error metrics also seem bleak. the model is not so successful. 

# next we train to predict obesity with walkability metrics

In [35]:
X = df_state[['D1B', 'R_PCTLOWWAGE', 'Pct_AO0', 'D4A', 'NatWalkInd']]
y = df_state['obesity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 1)

X_scaled = preprocessing.StandardScaler().fit_transform(X_train)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [36]:
mae = mean_absolute_error(y_test, y_pred)
mae
mse = mean_squared_error(y_test, y_pred)
mse
r2 = r2_score(y_test, y_pred)
r2
print(f'MAE: {mae}, MSE: {mse}, R2: {r2}')

MAE: 2.0674005841905467, MSE: 5.7477565320820885, R2: 0.49114679830884156


In [37]:
print(f'Model coefficients:\n{model.coef_}\n intercept:\n{model.intercept_}')

Model coefficients:
[ 1.51051670e-01  9.17670081e+01 -7.81816397e+00  3.14789179e-03
 -7.62167931e-01]
 intercept:
15.708642054255332
